In [1]:
! pip install pyspark


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [1]:

from pyspark.sql.types import *
from pyspark.sql.window import Window

import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
      .config("hive.metastore.uris", "thrift://172.27.1.8:9083")\
      .config("spark.sql.warehouse.dir", "/user/hive/warehouse")\
      .enableHiveSupport() \
      .getOrCreate() 

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/09 14:48:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/09 14:49:00 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [14]:
spark.sql("select * from retail_db.orders_avro").show()

+-----------------+-------------+--------+---------------+
|order_customer_id|   order_date|order_id|   order_status|
+-----------------+-------------+--------+---------------+
|            11599|1374710400000|       1|         CLOSED|
|              256|1374710400000|       2|PENDING_PAYMENT|
|            12111|1374710400000|       3|       COMPLETE|
|             8827|1374710400000|       4|         CLOSED|
|            11318|1374710400000|       5|       COMPLETE|
|             7130|1374710400000|       6|       COMPLETE|
|             4530|1374710400000|       7|       COMPLETE|
|             2911|1374710400000|       8|     PROCESSING|
|             5657|1374710400000|       9|PENDING_PAYMENT|
|             5648|1374710400000|      10|PENDING_PAYMENT|
|              918|1374710400000|      11| PAYMENT_REVIEW|
|             1837|1374710400000|      12|         CLOSED|
|             9149|1374710400000|      13|PENDING_PAYMENT|
|             9842|1374710400000|      14|     PROCESSIN

In [16]:
spark.sql('Select * from retail_db.customers_avro').show()

+-------------+--------------+--------------+-----------+--------------+-----------------+--------------+--------------------+----------------+
|customer_city|customer_email|customer_fname|customer_id|customer_lname|customer_password|customer_state|     customer_street|customer_zipcode|
+-------------+--------------+--------------+-----------+--------------+-----------------+--------------+--------------------+----------------+
|  Brownsville|     XXXXXXXXX|       Richard|          1|     Hernandez|        XXXXXXXXX|            TX|  6303 Heather Plaza|           78521|
|    Littleton|     XXXXXXXXX|          Mary|          2|       Barrett|        XXXXXXXXX|            CO|9526 Noble Embers...|           80126|
|       Caguas|     XXXXXXXXX|           Ann|          3|         Smith|        XXXXXXXXX|            PR|3422 Blue Pioneer...|           00725|
|   San Marcos|     XXXXXXXXX|          Mary|          4|         Jones|        XXXXXXXXX|            CA|  8324 Little Common|          

In [3]:
spark.sql("select * from retail_db.customers_avro").show(truncate=False)

24/06/09 14:49:20 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


+-------------+--------------+--------------+-----------+--------------+-----------------+--------------+---------------------------+----------------+
|customer_city|customer_email|customer_fname|customer_id|customer_lname|customer_password|customer_state|customer_street            |customer_zipcode|
+-------------+--------------+--------------+-----------+--------------+-----------------+--------------+---------------------------+----------------+
|Brownsville  |XXXXXXXXX     |Richard       |1          |Hernandez     |XXXXXXXXX        |TX            |6303 Heather Plaza         |78521           |
|Littleton    |XXXXXXXXX     |Mary          |2          |Barrett       |XXXXXXXXX        |CO            |9526 Noble Embers Ridge    |80126           |
|Caguas       |XXXXXXXXX     |Ann           |3          |Smith         |XXXXXXXXX        |PR            |3422 Blue Pioneer Bend     |00725           |
|San Marcos   |XXXXXXXXX     |Mary          |4          |Jones         |XXXXXXXXX        |CA  

In [4]:
spark.sql("select * from retail_db.departments_avro").show(truncate=False)

+-------------+---------------+
|department_id|department_name|
+-------------+---------------+
|2            |Fitness        |
|3            |Footwear       |
|4            |Apparel        |
|5            |Golf           |
|6            |Outdoors       |
|7            |Fan Shop       |
+-------------+---------------+



In [5]:
spark.sql("select * from retail_db.orders_avro").show(truncate=False)

+-----------------+-------------+--------+---------------+
|order_customer_id|order_date   |order_id|order_status   |
+-----------------+-------------+--------+---------------+
|11599            |1374710400000|1       |CLOSED         |
|256              |1374710400000|2       |PENDING_PAYMENT|
|12111            |1374710400000|3       |COMPLETE       |
|8827             |1374710400000|4       |CLOSED         |
|11318            |1374710400000|5       |COMPLETE       |
|7130             |1374710400000|6       |COMPLETE       |
|4530             |1374710400000|7       |COMPLETE       |
|2911             |1374710400000|8       |PROCESSING     |
|5657             |1374710400000|9       |PENDING_PAYMENT|
|5648             |1374710400000|10      |PENDING_PAYMENT|
|918              |1374710400000|11      |PAYMENT_REVIEW |
|1837             |1374710400000|12      |CLOSED         |
|9149             |1374710400000|13      |PENDING_PAYMENT|
|9842             |1374710400000|14      |PROCESSING    

In [8]:
spark.sql("select * from retail_db.order_items_avro").show(truncate=False)

+-------------+-------------------+---------------------+------------------------+-------------------+-------------------+
|order_item_id|order_item_order_id|order_item_product_id|order_item_product_price|order_item_quantity|order_item_subtotal|
+-------------+-------------------+---------------------+------------------------+-------------------+-------------------+
|1            |1                  |957                  |299.98                  |1                  |299.98             |
|2            |2                  |1073                 |199.99                  |1                  |199.99             |
|3            |2                  |502                  |50.0                    |5                  |250.0              |
|4            |2                  |403                  |129.99                  |1                  |129.99             |
|5            |4                  |897                  |24.99                   |2                  |49.98              |
|6            |4

In [9]:
spark.sql("select * from retail_db.products_avro").show(truncate=False)

+-------------------+-------------------+----------+------------------------------------------------------------------------------------------------+---------------------------------------------+-------------+
|product_category_id|product_description|product_id|product_image                                                                                   |product_name                                 |product_price|
+-------------------+-------------------+----------+------------------------------------------------------------------------------------------------+---------------------------------------------+-------------+
|2                  |                   |1         |http://images.acmesports.sports/Quest+Q64+10+FT.+x+10+FT.+Slant+Leg+Instant+Up+Canopy           |Quest Q64 10 FT. x 10 FT. Slant Leg Instant U|59.98        |
|2                  |                   |2         |http://images.acmesports.sports/Under+Armour+Men%27s+Highlight+MC+Football+Cleat                |Under Armou

### Enunciado 1
Mostrar un Top 20 de clientes que mas productos compraron con sus respectivos montos.

In [6]:
spark.sql("""SELECT
    customer_id, customer_fname, customer_lname, customer_email, sum(order_item_quantity) as quantity_item_total, sum(order_item_subtotal)as total
FROM
    retail_db.customers_avro as c
INNER JOIN
    retail_db.orders_avro as o
    ON c.customer_id = o.order_customer_id
INNER JOIN
    retail_db.order_items_avro as oi
    ON o.order_id = oi.order_item_order_id
WHERE order_status <> 'CANCELED'
GROUP BY customer_id, customer_fname, customer_lname, customer_email
ORDER BY  total DESC
LIMIT 20""").show()

+-----------+--------------+--------------+--------------+-------------------+------------------+
|customer_id|customer_fname|customer_lname|customer_email|quantity_item_total|             total|
+-----------+--------------+--------------+--------------+-------------------+------------------+
|        791|          Mary|         Smith|     XXXXXXXXX|                 82|10524.170177459717|
|       8766|          Mary|        Duncan|     XXXXXXXXX|                 87| 9296.140186309814|
|       1657|         Betty|      Phillips|     XXXXXXXXX|                111| 9223.710151672363|
|       2641|         Betty|        Spears|     XXXXXXXXX|                 90| 9130.920223236084|
|       1288|        Evelyn|      Thompson|     XXXXXXXXX|                 89|  9019.11019897461|
|       3710|        Ashley|         Smith|     XXXXXXXXX|                 89|  9019.10020828247|
|       5654|         Jerry|         Smith|     XXXXXXXXX|                103| 8904.950210571289|
|       5624|       

### Enunciado 2
Mostrar las categorías con el total de productos vendidos y los montos totales por categoría

In [7]:
spark.sql("""SELECT
    ca.category_name, sum(order_item_quantity) as item_quantity, cast(sum(order_item_subtotal) AS INT )as total
FROM retail_db.order_items_avro as oi
INNER JOIN
    retail_db.products_avro as p
    ON oi.order_item_product_id = p.product_id
INNER JOIN
    retail_db.categories_avro as ca
    ON p.product_category_id = ca.category_id
GROUP BY ca.category_name;
""").show()

+-------------------+-------------+-------+
|      category_name|item_quantity|  total|
+-------------------+-------------+-------+
|   Camping & Hiking|        13729|4118425|
|Fitness Accessories|          856|  35601|
|         Golf Shoes|         1444| 107998|
|     Men's Footwear|        22246|2891757|
|        Electronics|         9436| 371034|
|    Women's Apparel|        62956|3147800|
|     Girls' Apparel|         3615| 151706|
|       Boxing & MMA|         1265|  85205|
|Baseball & Softball|         1785|  94057|
| Hunting & Shooting|         1239|  56848|
|       Golf Apparel|         1025|  34969|
|           Trade-In|         2822|  68721|
|      Shop By Sport|        32726|1309522|
|            Fishing|        17325|6929653|
|       Water Sports|        15540|3113844|
|        Accessories|         5349| 133671|
|        Golf Gloves|         2961| 116695|
|             Cleats|        73734|4431942|
|         Golf Balls|         4384|  77098|
|             Hockey|         17

### Enunciado 3
Mostrar la categoría más vendida por ciudad

In [8]:
spark.sql("""
SELECT
    customer_city, category_name
FROM (SELECT
    customer_city, category_name, count(category_name) as quantity, DENSE_RANK () OVER ( 
                PARTITION BY customer_city 
                ORDER BY count(category_name) DESC
            ) rank
    FROM
        retail_db.customers_avro as c
    INNER JOIN
        retail_db.orders_avro as o
        ON c.customer_id = o.order_customer_id
    INNER JOIN
        retail_db.order_items_avro as oi
        ON o.order_id = oi.order_item_order_id
    INNER JOIN
        retail_db.products_avro as p
        ON oi.order_item_product_id = p.product_id
    INNER JOIN
        retail_db.categories_avro as ca
        ON p.product_category_id = ca.category_id
    GROUP BY customer_city, category_name
    ) t
WHERE rank = 1;""").show()

+-----------------+--------------------+
|    customer_city|       category_name|
+-----------------+--------------------+
|        Aguadilla|              Cleats|
|          Alameda|              Cleats|
|           Albany|     Women's Apparel|
|      Albuquerque|              Cleats|
|        Algonquin|              Cleats|
|         Alhambra|     Women's Apparel|
|        Allentown|              Cleats|
|       Alpharetta|      Men's Footwear|
|         Amarillo|              Cleats|
|         Amarillo|Indoor/Outdoor Games|
|          Anaheim|     Women's Apparel|
|        Ann Arbor|     Women's Apparel|
|        Annandale|      Men's Footwear|
|        Annapolis|      Men's Footwear|
|          Antioch|Indoor/Outdoor Games|
|             Apex|              Cleats|
|           Apopka|             Fishing|
|          Arecibo|     Women's Apparel|
|        Arlington|     Women's Apparel|
|Arlington Heights|     Women's Apparel|
+-----------------+--------------------+
only showing top

### Enunciado 4
Mostrar los 5 productos más vendidos por cada ciudad y el monto recaudado

In [13]:
spark.sql("""
SELECT
    customer_city, product_name, quantity, total
FROM (SELECT
    customer_city, product_name,sum(order_item_quantity) as quantity,sum(order_item_subtotal) as total, DENSE_RANK () OVER ( 
                PARTITION BY customer_city 
                ORDER BY sum(order_item_quantity) DESC
            ) rank
    FROM
        retail_db.customers_avro as c
    INNER JOIN
        retail_db.orders_avro as o
        ON c.customer_id = o.order_customer_id
    INNER JOIN
        retail_db.order_items_avro as oi
        ON o.order_id = oi.order_item_order_id
    INNER JOIN
        retail_db.products_avro as p
        ON oi.order_item_product_id = p.product_id
    GROUP BY customer_city, product_name
    ) t
WHERE rank < 6
ORDER BY quantity DESC;
""").show()

+-------------+--------------------+--------+------------------+
|customer_city|        product_name|quantity|             total|
+-------------+--------------------+--------+------------------+
|       Caguas|Perfect Fitness P...|   27276|1636287.2855148315|
|       Caguas|Nike Men's Dri-FI...|   23355|         1167750.0|
|       Caguas|O'Brien Men's Neo...|   21590|1079068.1897773743|
|       Caguas|Nike Men's Free 5...|   13530|1352864.6990203857|
|       Caguas|Under Armour Girl...|   11713| 468402.8778114319|
|      Chicago|Perfect Fitness P...|    1540| 92384.60255813599|
|  Los Angeles|Perfect Fitness P...|    1423| 85365.77238464355|
|      Chicago|Nike Men's Dri-FI...|    1391|           69550.0|
|      Chicago|O'Brien Men's Neo...|    1278| 63874.43938827515|
|     Brooklyn|Perfect Fitness P...|    1251| 75047.49206924438|
|     Brooklyn|Nike Men's Dri-FI...|    1237|           61850.0|
|  Los Angeles|Nike Men's Dri-FI...|    1194|           59700.0|
|     Brooklyn|O'Brien Me